## Input Variables Processing 

Input variables processing converts raw data to usable information and speeds up modeling/analytic processes. In this notebook,  input variables of battery forecasting competition at Kaggle is processed (https://www.kaggle.com/competitions/eso-battery-forecasting/data). 

The process and outcome of the following four steps are illustrated. 

1. Missing Values
    - Identify columns which have missing values

2. Columns types: Depending on number of unique values of each column, they can be divided into three groups:

    - Constant value columns- column has only one value for all rows
    - Categorical columns - column has small number (< 5) of unique values. Further, they can be either numerical and object data types.
    - Continuous columns - numerical values

3. Data impute: Based on column types, imputation method is decided.

    - Determine methods of imputing missing values columns
    - After imputation, check number of null values and decide the action

4. Data Cleaning

    - Drop columns and rows of less or no relevance 

In [1]:
import pandas as pd

from TimeindexProcessing import TimeindexProcessing
from InputVariablesProcessing import InputVariablesProcessing

In [2]:
df = pd.read_csv('./Data/Kaggle_ESO_battery/train_data.csv')
df.head()

,id,UTC_Settlement_DateTime,battery_output,East_Midlands_price,Eastern_England_price,London_price,Merseyside_and_Northern_Wales_price,North_Western_England_price,Northern_Scotland_price,South_Western_England_price,...,is_daySouthhampton - Portsmouth_weather,locationSouthhampton - Portsmouth_weather,populationSouthhampton - Portsmouth_weather,latitudeSouthhampton - Portsmouth_weather,longitudeSouthhampton - Portsmouth_weather,generationtime_msSouthhampton - Portsmouth_weather,utc_offset_secondsSouthhampton - Portsmouth_weather,timezoneSouthhampton - Portsmouth_weather,timezone_abbreviationSouthhampton - Portsmouth_weather,elevationSouthhampton - Portsmouth_weather
0,0,8/10/2020 23:30,1.528,5.985,6.279,5.985,6.5835,6.279,7.182,6.888,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,8/11/2020 0:00,0.178,6.300,6.615,6.300,6.9300,6.615,7.560,7.245,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
2,2,8/11/2020 0:30,-1.242,6.300,6.615,6.300,6.9300,6.615,7.560,7.245,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,8/11/2020 1:00,2.104,5.985,6.279,5.985,6.5835,6.279,7.182,6.888,...,0.0,Southhampton - Portsmouth,927916.0,50.699997,-1.199997,61.430931,0.0,GMT,GMT,10.0
4,4,8/11/2020 1:30,-0.664,5.985,6.279,5.985,6.5835,6.279,7.182,6.888,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Time Index Processing**

By using the developed class of time index processing, 
1. settlement date and time is converted into datetime index.
2. Checked duplicates time index and create new dataframe while keeping only first of duplicates 
3. Find out any missing time index and add them in dataframe.

In [3]:
# Create the class object
index_processing = TimeindexProcessing()

# Convert a column 'datetime' into time index
indexed_df = index_processing.convert_column_to_timeindex(df, column_name= 'UTC_Settlement_DateTime')

# Determine a list of duplicates index and build a new dataframe by keeping only first row of duplicates index
duplicates_index_list, duplicates_corrected_df = index_processing.duplicate_timeindex(indexed_df)

# Duplicates index in the original dataframe can be checked by:
indexed_df.loc[duplicates_index_list]

# Duplicates corrected dataframe for original dataframe's duplicates:
duplicates_corrected_df.loc[duplicates_index_list]

# Identify missing index based on declared data frequency and add these rows into duplicates corrected dataframe
data_freq = '0.5H' # Define the expected data frequency
missing_index_list, rows_added_df = index_processing.missing_timeindex(duplicates_corrected_df, data_freq)

# To confirm whether missing rows are added in dataframe:
# rows_added_df.loc[missing_index_list]

There are no duplicate time index
All time index present


Time index processed dataframe is splitted into a dataframe of input features and a series of output parameter. Both of these are analzyed in different ways. In the following section, we focus on analyzing data processing of input features. They are identifying misisng values columns, types of columns (categorical/continuous). Based on these information, data imputation and cleaning would be performed.

In [4]:
cleaned_index_df = rows_added_df.copy()
output_parameter = 'battery_output'
# Extract the output series from cleaned dataframe
output_series = cleaned_index_df[output_parameter]

# Input variables dataframe
input_features_df = cleaned_index_df.copy()
input_features_df.drop(output_parameter, axis = 1, inplace = True)

# Defining list of columns to drop and a dictionary of columns (key - method, value - list of columns) to impute 
# which will be appended based on outcome of each section
columns_to_drop = []
columns_to_impute = {}

### Missing Values

- First, the number of missing values of each column is identified. Either these columns are dropped or imputed. 
- For our data, weather columns have almost 50% missing values as they were measured at one hour interval while battery output is at half an hour interval. Therefore, weather columns can be filled by linear interpolation. 
- Other than weather parameters, west midlands price is missing for a day. This data can be filled by forward filling. 

In [5]:
ivp = InputVariablesProcessing(input_features_df)
missing_values_series = ivp.missing_rows_count()
missing_values_series

West_Midlands_price                                          44
temperature_2mBelfast_weather                             19729
relativehumidity_2mBelfast_weather                        19729
dewpoint_2mBelfast_weather                                19729
apparent_temperatureBelfast_weather                       19729
                                                          ...  
generationtime_msSouthhampton - Portsmouth_weather        19729
utc_offset_secondsSouthhampton - Portsmouth_weather       19729
timezoneSouthhampton - Portsmouth_weather                 19729
timezone_abbreviationSouthhampton - Portsmouth_weather    19729
elevationSouthhampton - Portsmouth_weather                19729
Length: 281, dtype: int64

In [6]:
weather_columns_to_impute = missing_values_series[missing_values_series >45].index.tolist()
price_columns_to_impute = ['West_Midlands_price']

In [14]:
# df[df['West_Midlands_price'].isna()]

### Columns types: categorical/continuous 

**Constant value columns**

In [7]:
constant_values_series = ivp.constant_values()
constant_values_series

locationBelfast_weather                                       [nan, Belfast]
populationBelfast_weather                                    [nan, 630632.0]
latitudeBelfast_weather                                     [nan, 54.600006]
longitudeBelfast_weather                                    [nan, -5.899994]
generationtime_msBelfast_weather                          [nan, 1807.518005]
                                                                 ...        
generationtime_msSouthhampton - Portsmouth_weather        [nan, 61.43093109]
utc_offset_secondsSouthhampton - Portsmouth_weather               [nan, 0.0]
timezoneSouthhampton - Portsmouth_weather                         [nan, GMT]
timezone_abbreviationSouthhampton - Portsmouth_weather            [nan, GMT]
elevationSouthhampton - Portsmouth_weather                       [nan, 10.0]
Length: 90, dtype: object

These column values might be useful for connecting other data source but not for modeling. Therefore, these columns are worth to drop.

In [8]:
columns_to_drop = columns_to_drop + constant_values_series.index.tolist()

**Small unique value columns**

In [9]:
small_uniquevalue_series = ivp.small_unique_values()
small_uniquevalue_series

is_dayBelfast_weather                      [nan, 0.0, 1.0]
is_dayBirmingham_weather                   [nan, 0.0, 1.0]
is_dayGlasgow_weather                      [nan, 0.0, 1.0]
is_dayLeeds_weather                        [nan, 0.0, 1.0]
is_dayLiverpool_weather                    [nan, 0.0, 1.0]
is_dayLondon_weather                       [nan, 0.0, 1.0]
is_dayManchester_weather                   [nan, 0.0, 1.0]
is_dayNewcastle upon Tyne_weather          [nan, 0.0, 1.0]
is_daySheffield_weather                    [nan, 0.0, 1.0]
is_daySouthhampton - Portsmouth_weather    [nan, 0.0, 1.0]
dtype: object

All of the above columns indicates whether it is a day time or night time. Therefore, these columns are grouped in categorical numerical values. They must be imputed by either back fill or forward fill.

In [10]:
categorical_numerical_columns = small_uniquevalue_series.index.tolist()

In [11]:
remaining_weather_columns = list(set(weather_columns_to_impute) - set(categorical_numerical_columns))
weather_object_columns = df[remaining_weather_columns].select_dtypes(include = 'object').columns.tolist()
weather_numerical_columns = df[remaining_weather_columns].select_dtypes(exclude = 'object').columns.tolist()

### Data Imputation

- Categorical columns (both object and distinct numeric values) are imputed by forward fill.
- Continuous numerical columns are imputed by linear interpolation.

In [12]:
# Categorical columns to be imputed by forward fill
columns_to_impute['ffill'] =  weather_object_columns + categorical_numerical_columns 
# Continuous columns to be imputed by linear interpolation
columns_to_impute['linear'] = weather_numerical_columns + price_columns_to_impute

imputed_df = ivp.data_imputation(input_features_df, columns_to_impute)

*Last check on missing values*

In [13]:
rows_missing_values = imputed_df.isna().sum(axis = 1)
rows_missing_values[rows_missing_values >0]

UTC_Settlement_DateTime
2020-08-10 23:30:00    280
dtype: int64

After data imputation, only the first row has null values. Therefore this row can be dropped.

In [14]:
rows_to_drop = rows_missing_values[rows_missing_values >0].index.tolist()
rows_to_drop

[Timestamp('2020-08-10 23:30:00')]

**Data Cleaning**

Dropping rows and columns which were determined of less relevance from the above data processing.

In [15]:
print('Raw data shape:', df.shape)
print('Imputed df shape:', imputed_df.shape)
cleaned_df = ivp.data_cleaning(imputed_df, columns_to_drop = columns_to_drop, rows_to_drop = rows_to_drop)
print('Cleaned (after dropping less relevant rows and columns)', cleaned_df.shape)

Raw data shape: (39457, 294)
Imputed df shape: (39457, 292)
Cleaned (after dropping less relevant rows and columns) (39456, 202)
